In [2]:
import pandas as pd
from preprocess import *
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy
from IPython.display import Image

In [3]:
data=pd.read_csv('c_employment.csv')

data['c_employment_job_end']=data['c_employment_job_end'].apply(lambda t: datetime.now().strftime("%Y-%m-%d") if t=='present' else t )

data['c_employment_job_end'] = data['c_employment_job_end'].map(clean_datetime)
data['c_employment_job_start'] = data['c_employment_job_start'].map(clean_datetime)

data['c_employment_working']=data['c_employment_job_end']-data['c_employment_job_start']

data['c_employment_working']=data['c_employment_working'].apply(lambda t: numpy.ceil(float(t.days)/365))




<img src="untitled.png">

In [4]:
#define some keywords to determine the label

production=['production','product','intern']
business_support=['business','support','supporter','supervisor']
professional_expert=['professional','expert','specialist','master']
management=['management','manager']
executive=['executive','president','chief','director']


In [5]:
data['l_production']=data['c_employment_title'].apply(lambda t: 1 if (any(x.upper() in t.upper() for x in production)) else 0)
data['l_business']=data['c_employment_title'].apply(lambda t: 1 if (any(x.upper() in t.upper() for x in business_support)) else 0)
data['l_expert']=data['c_employment_title'].apply(lambda t: 1 if (any(x.upper() in t.upper() for x in professional_expert)) else 0)
data['l_manager']=data['c_employment_title'].apply(lambda t: 1 if (any(x.upper() in t.upper() for x in management)) else 0)
data['l_executive']=data['c_employment_title'].apply(lambda t: 1 if (any(x.upper() in t.upper() for x in executive)) else 0)

df = data[['c_employment_title','c_employment_working','l_production','l_business','l_expert','l_manager','l_executive']]


In [6]:
label=[]
index=0
for title in df['c_employment_title']:
    result='Unspecified'
    if df['l_production'][index]!=0:
        result='Production'
    if df['l_business'][index]!=0:
        result='Business Support'
    if df['l_expert'][index]!=0:
        result='Professional Expert'
    if df['l_manager'][index]!=0:
        result='Management'
    if df['l_executive'][index]!=0:
        result='Executive'
    label.append(result)
    index=index+1
df['label']=label

C:\Users\TranBinh\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#define some working experience

production_l=[1,2,3] # less than 1  year=> entry, 1<x<2: intermediate, 2<x<3: senior
business_l=[1,2,3,5]
professional_l=[1,2,3,5,10]
manager_l=[1,2,3,5,10]
executive_l=[1,2,3]


In [8]:
def sub_label(data):
    s_label=[]
    for i in range(0, len(data)):
        sub_label=''
        
        if (data['label'][i]=='Production'):
            if data['c_employment_working'][i]<=production_l[0]:
                sub_label='Entry'
            elif data['c_employment_working'][i]<=production_l[1]:
                sub_label='Intermediate'
            elif data['c_employment_working'][i]<=production_l[2]:
                sub_label='Senior'
            else: sub_label='Lead'
        
        if (data['label'][i]=='Business Support'):
            if data['c_employment_working'][i]<=business_l[0]:
                sub_label='Entry'
            elif data['c_employment_working'][i]<=business_l[1]:
                sub_label='Intermediate'
            elif data['c_employment_working'][i]<=business_l[2]:
                sub_label='Senior'
            elif data['c_employment_working'][i]<=business_l[3]:
                sub_label='Lead'
            else: sub_label='Supervisor'
        
        if (data['label'][i]=='Professional Expert'):
            if data['c_employment_working'][i]<=professional_l[0]:
                sub_label='Entry'
            elif data['c_employment_working'][i]<=professional_l[1]:
                sub_label='Intermediate'
            elif data['c_employment_working'][i]<=professional_l[2]:
                sub_label='Career'
            elif data['c_employment_working'][i]<=professional_l[3]:
                sub_label='Specialist'
            elif data['c_employment_working'][i]<=professional_l[4]:
                sub_label='Master'
            else: sub_label='Expert'
            
        if (data['label'][i]=='Management'):
            if data['c_employment_working'][i]<=manager_l[0]:
                sub_label='Supervisor'
            elif data['c_employment_working'][i]<=manager_l[1]:
                sub_label='Manager'
            elif data['c_employment_working'][i]<=manager_l[2]:
                sub_label='Senior Manager'
            elif data['c_employment_working'][i]<=manager_l[3]:
                sub_label='Group Manager'
            else: sub_label='Senior Group Manager'
    
        if (data['label'][i]=='Executive'):
            if data['c_employment_working'][i]<=executive_l[0]:
                sub_label='Vice President'
            elif data['c_employment_working'][i]<=executive_l[1]:
                sub_label='Senior Vice President'
            elif data['c_employment_working'][i]<=executive_l[2]:
                sub_label='Executive Vice President'
            else: sub_label='Chief Executive Officer'
            
        
        s_label.append(sub_label)
    return s_label
        

In [9]:
df['sub_label']=sub_label(df)

df.to_csv('output.csv',sep=',')

C:\Users\TranBinh\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
